# Preparation for elastic Net Prediction

Extracting of FC feature values and elastic net prediction of 24h post MRS values

In [3]:
import numpy as np
import pickle
import pandas as pd
import os

In [4]:
# lists with subjects in conditions
subj_k= np.load('C:/Users/canlab/Desktop/Masterthesis_python/New_ketamine_subject_list', allow_pickle=True)
subj_p= np.load('C:/Users/canlab/Desktop/Masterthesis_python/New_placebo_subject_list', allow_pickle=True)

In [5]:
subj_k_df=pd.DataFrame(subj_k)
subj_p_df=pd.DataFrame(subj_p)

Saving subject lists as csv

In [6]:
subj_k_df.to_csv('C:/Users/canlab/Desktop/Masterthesis_python/New_ketamine_subject_list.csv')
subj_p_df.to_csv('C:/Users/canlab/Desktop/Masterthesis_python/New_placebo_subject_list.csv')

In [7]:
subj_p_df

,0
0,ap75
1,bu45
2,dp20
3,dp92
4,dq56
5,dt27
6,du02
7,dw28
8,em60
9,ew27


In [18]:
subj_k

0     bg52
1     bh98
2     ce22
3     cu52
4     dp07
5     dp37
7     fy78
8     gs63
9     hw95
10    hy97
11    ia24
12    ia45
13    ii39
14    il04
15    ju09
16    ju49
17    jw73
18    jy99
19    kh95
20    kw91
21    op18
22    px00
23    qe30
24    sr21
25    tt42
26    uf97
27    vg84
28    we81
29    wo30
30    ww25
31    yp84
Name: 0, dtype: object

In [6]:
# 5 voxels each with largest change induced by ketamine
s24 = np.load('C:/Users/canlab/Desktop/Masterthesis_python/largest_5_voxels_s24', allow_pickle=True)
s32 = np.load('C:/Users/canlab/Desktop/Masterthesis_python/largest_5_voxels_s32', allow_pickle=True)

In [17]:
# voxel labels & IDs
labels = np.load('C:/Users/canlab/Desktop/Masterthesis_python/Voxel_ID_ROI_Membership', allow_pickle=True)

In [7]:
s24

,x,y,ROI,ID,Dist,Diff,Diff_k_p
4877,-3.058445,-7.782920,s24,314453,3.937080,0.594926,8.236997
5373,-3.137073,-7.780452,s24,324372,3.963489,0.617523,7.804681
5379,-3.092263,-7.514478,s24,324463,3.700498,0.533469,7.471633
4870,-2.927916,-7.864942,s24,314363,3.970085,0.022655,6.976734
4257,0.408622,0.971157,s24,304349,5.479118,4.179372,5.778891


In [50]:
s32

,x,y,ROI,ID,Dist,Diff,Diff_k_p
2331,0.019293,-0.359633,s32,274771,4.069439,3.511681,4.133112
2335,-0.003465,-0.409194,s32,274776,4.014970,3.086293,4.009193
2304,0.052073,-0.277591,s32,274680,4.157788,3.392876,3.744995
2332,0.012188,-0.348078,s32,274772,4.077534,3.395457,3.698203
2359,-0.050799,-0.427927,s32,274862,3.980160,3.395256,3.628615


In [ ]:
# need to to read in participant FC matrices: ket/placebo and its baseline
# ket/placebo - baseline
# median of ROIs to the 10 voxels
# so for each participant you get an 10 x 18 matrix: difference between ket/placebo & its baseline
# then look into MRS values 24h after and look at elastic net prediction


In [61]:
path_ket = 'C:/Users/canlab/Desktop/Masterthesis_python/ket/'
path_placebo = 'C:/Users/canlab/Desktop/Masterthesis_python/placebo/'
path_baseline = 'C:/Users/canlab/Desktop/Masterthesis_python/baseline'

In [67]:
# output: 18 x 10 matrix for each participant
def calculate_FC_features(subj_list, path_1h, path_b):
    
    file_prefix_1h = 'corr_mat_all_ses-1h_sub-'
    file_prefix_b = 'corr_mat_all_ses-baseline_sub-'
    
    for subj in subj_list:
        # filename 1h and baseline
        file_name_1h = file_prefix_1h + subj + '.csv'
        file_name_b = file_prefix_b + subj + '.csv'

        # path of the current subject
        curr_subj_1h_path = os.path.join(path_1h, file_name_1h)
        curr_subj_b_path = os.path.join(path_b, file_name_b)

        # read in current subject as dataframe
        curr_subj_1h = pd.read_csv(curr_subj_1h_path, header=None)
        curr_subj_b = pd.read_csv(curr_subj_b_path, header=None)
        
        # subtract the baseline from the 1h condition to get the difference
        subj_diff = curr_subj_1h.subtract(curr_subj_b)
        
        # concatenate subject FC and labels+ID
        subj_diff_conc =pd.concat([subj_diff, labels['Label']], axis=1)
        
        display(subj_diff_conc)
        
        # split according to ROIs
        split_df = subj_diff_conc.groupby('Label')
        # get the median correlation of the ROIs
        median_df = split_df.median()
        
        # extracting the s24 and s32 voxels
        # first 5: s24 and last 5: s32
        new_df = median_df[[4877, 5373, 5379, 4870, 4257, 2331, 2335, 2304, 2332, 2359]].copy()
        display(new_df)
 
        save_path = path_1h + 'FC_features-' + subj
        new_df.to_pickle(save_path)
        
        del curr_subj_1h, curr_subj_b, median_df, new_df
    

Calculating FC features for ketamine and placebo 

In [68]:
calculate_FC_features(subj_k, path_ket, path_baseline)

,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,0.127854,0.065643,-0.198610,0.027552,-0.056385,0.216656,-0.110571,-0.011132,-0.052437,...,-0.069096,-0.018717,-0.067031,-0.042474,-0.120957,-0.078146,-0.195982,-0.123966,-0.003212,OFC1
1,0.127854,0.000000,0.182837,-0.202029,-0.044006,-0.195576,0.258397,-0.158951,0.168185,0.285810,...,0.095840,-0.047379,0.182377,-0.059005,0.096099,-0.004451,-0.109167,-0.144505,-0.042608,OFC1
2,0.065643,0.182837,0.000000,0.074905,-0.054351,-0.056633,0.285743,0.020664,0.278985,0.146274,...,0.020788,-0.019202,0.219870,-0.131546,-0.087244,-0.119970,-0.139639,-0.128659,-0.121037,OFC1
3,-0.198610,-0.202029,0.074905,0.000000,0.254485,0.055286,-0.185289,-0.021955,0.161133,0.050057,...,0.169684,-0.011458,0.137358,0.034070,-0.059499,0.128377,-0.007281,-0.047889,-0.039924,OFC1
4,0.027552,-0.044006,-0.054351,0.254485,0.000000,-0.061558,0.167968,-0.141910,0.058558,0.210897,...,0.122320,-0.063164,0.134331,0.024741,0.003810,0.033972,-0.026390,-0.073707,-0.030683,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.120957,0.096099,-0.087244,-0.059499,0.003810,-0.201277,0.082623,0.013880,0.157791,-0.049192,...,0.041045,0.094281,0.149966,-0.046263,0.000000,0.005190,0.041262,-0.023612,-0.102716,Caudate
11879,-0.078146,-0.004451,-0.119970,0.128377,0.033972,-0.249385,0.104246,-0.058989,0.028369,-0.077504,...,-0.068152,0.161711,-0.083131,0.023192,0.005190,0.000000,-0.012522,-0.062270,-0.015150,Caudate
11880,-0.195982,-0.109167,-0.139639,-0.007281,-0.026390,-0.230870,0.140254,-0.047179,-0.023253,-0.047538,...,-0.070207,0.028224,-0.095649,-0.075706,0.041262,-0.012522,0.000000,-0.104398,-0.090457,Caudate
11881,-0.123966,-0.144505,-0.128659,-0.047889,-0.073707,-0.242871,-0.024613,-0.000698,0.015974,-0.120083,...,0.085553,-0.023581,0.026828,-0.062846,-0.023612,-0.062270,-0.104398,0.000000,-0.105543,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.060794,-0.039162,-0.030193,-0.030875,-0.006244,-0.014368,0.109128,-0.069767,0.002359,-0.037030
33,-0.016118,-0.010245,-0.008499,-0.015682,-0.025337,-0.018579,0.016135,0.019050,-0.019523,0.010395
Caudate,0.018753,-0.004139,-0.005690,0.010057,-0.006822,-0.001468,-0.045082,0.024679,0.032477,-0.012345
FP2,0.041885,-0.040878,0.020763,0.027150,0.022503,-0.061360,-0.057410,0.040132,-0.044452,-0.039697
NAcc,-0.038510,-0.050074,-0.025111,-0.009234,0.024019,0.023853,0.017390,-0.001289,0.024356,-0.024291
OFC1,-0.095958,-0.063177,0.011704,-0.090524,0.007438,0.051886,0.117975,-0.033560,-0.025783,-0.001090
OFC2,-0.009966,-0.020165,0.011647,-0.033929,-0.009588,0.008262,0.064357,-0.018491,-0.016428,-0.017696
OFC3,-0.020115,-0.026960,-0.001697,-0.026784,-0.002077,0.016961,0.052406,-0.012243,-0.004439,-0.009416
OFC4,-0.071011,-0.023258,-0.043145,-0.025431,0.019256,0.048199,0.058971,-0.048058,0.005126,-0.028535


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,0.419730,0.370536,0.389313,0.369771,-0.212896,-0.259359,-0.188097,-0.045421,-0.097857,...,-0.060813,-0.129132,0.165729,0.029946,-0.126039,-0.133566,-0.036140,-0.047221,-0.020322,OFC1
1,0.419730,0.000000,0.350369,0.594651,0.114651,-0.063755,-0.174784,-0.163509,-0.226438,-0.302579,...,-0.019673,-0.142904,0.126304,0.191681,0.092555,0.110355,0.166215,0.153636,0.075652,OFC1
2,0.370536,0.350369,0.000000,0.422355,0.346199,-0.130884,-0.113226,-0.211925,-0.249854,-0.206264,...,-0.029823,-0.113992,0.107673,0.129602,0.129816,0.126055,-0.011471,-0.146071,-0.167870,OFC1
3,0.389313,0.594651,0.422355,0.000000,0.303442,-0.148026,-0.253559,-0.411187,-0.202253,-0.205604,...,0.049126,-0.175248,0.121279,0.054271,0.097044,0.082530,-0.090262,-0.049539,-0.048294,OFC1
4,0.369771,0.114651,0.346199,0.303442,0.000000,0.027538,-0.209849,-0.227314,-0.329045,-0.176129,...,-0.196361,-0.299028,0.008405,-0.037726,0.161964,-0.013493,-0.043866,-0.022847,-0.042944,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.126039,0.092555,0.129816,0.097044,0.161964,-0.083290,-0.234093,-0.055613,0.043626,-0.010477,...,-0.121246,-0.084769,0.063279,0.123615,0.000000,0.173831,-0.009746,-0.032846,0.023482,Caudate
11879,-0.133566,0.110355,0.126055,0.082530,-0.013493,-0.049279,-0.235660,0.097960,0.043415,-0.048936,...,-0.598590,-0.884112,-0.664824,-0.789514,0.173831,0.000000,-0.238072,-0.085150,0.108762,Caudate
11880,-0.036140,0.166215,-0.011471,-0.090262,-0.043866,0.040695,-0.057437,0.123805,0.057015,0.048948,...,-0.636257,-0.813760,-0.671439,-0.782592,-0.009746,-0.238072,0.000000,0.147074,0.249016,Caudate
11881,-0.047221,0.153636,-0.146071,-0.049539,-0.022847,0.203663,0.128057,-0.080858,0.055857,0.031953,...,-0.260687,-0.224689,-0.150277,-0.133338,-0.032846,-0.085150,0.147074,0.000000,0.097557,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.006298,-0.025500,-0.013911,0.057999,-0.031943,0.018782,0.031207,0.061286,0.047726,0.053467
33,0.000933,0.013071,-0.008555,0.024007,-0.006916,0.000268,0.001036,-0.021622,0.017259,-0.003182
Caudate,-0.017500,0.008781,-0.023255,0.045702,0.027985,0.043505,0.050021,-0.000823,0.049505,0.025168
FP2,0.013603,0.012726,-0.093253,0.069012,0.059281,-0.018996,-0.010860,-0.057263,0.017124,-0.004596
NAcc,-0.140675,-0.009459,-0.050641,-0.021169,-0.013842,0.115845,0.120845,0.035414,0.070710,0.029873
OFC1,-0.079533,-0.014869,-0.046242,0.001599,-0.034982,0.043544,0.052861,0.068503,0.047303,0.053263
OFC2,-0.030223,0.004462,0.013184,0.004174,-0.049040,0.007884,0.027187,0.042776,0.031280,0.023913
OFC3,-0.042494,-0.002969,-0.029858,-0.004809,-0.011887,0.004003,0.017844,0.020171,0.035992,0.030405
OFC4,-0.077143,-0.014163,-0.056688,0.015785,0.079626,0.027472,0.032702,-0.008108,-0.003350,0.010480


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,-0.031275,-0.163602,-0.108572,-0.013673,-0.034932,-0.293105,0.166351,-0.294062,-0.174105,...,0.141712,0.128186,0.221911,-0.008670,0.133701,0.051558,0.169608,0.255320,0.044330,OFC1
1,-0.031275,0.000000,0.113263,0.203990,0.129531,0.043426,0.100575,-0.006180,-0.087133,-0.095342,...,0.139874,-0.076172,0.145469,-0.133480,0.285476,0.235525,0.094750,0.154169,0.094412,OFC1
2,-0.163602,0.113263,0.000000,0.262790,0.154604,0.004363,0.219960,-0.029982,0.140930,0.042120,...,0.089444,0.003907,0.094471,0.038255,0.087319,0.026960,-0.014256,0.058127,-0.000163,OFC1
3,-0.108572,0.203990,0.262790,0.000000,0.028187,-0.144009,0.198582,-0.069301,0.111037,0.010209,...,0.080559,-0.037497,-0.019532,0.099110,-0.010827,-0.030004,-0.089075,-0.056648,-0.073690,OFC1
4,-0.013673,0.129531,0.154604,0.028187,0.000000,0.175414,0.280378,0.023324,0.110303,-0.032860,...,-0.078760,-0.077818,-0.156744,0.042747,-0.025377,-0.132701,-0.193676,-0.196866,-0.191134,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,0.133701,0.285476,0.087319,-0.010827,-0.025377,0.083971,0.019598,-0.016782,-0.143054,0.042173,...,-0.318126,0.002681,-0.234584,0.075594,0.000000,-0.331067,-0.398377,-0.283147,-0.110870,Caudate
11879,0.051558,0.235525,0.026960,-0.030004,-0.132701,-0.039315,0.080134,-0.063919,-0.039942,0.016484,...,-0.204435,0.359650,-0.228362,0.175390,-0.331067,0.000000,-0.097318,-0.234968,-0.065123,Caudate
11880,0.169608,0.094750,-0.014256,-0.089075,-0.193676,-0.098966,0.110139,-0.129941,0.030593,0.181628,...,-0.172585,0.293437,-0.209992,0.207715,-0.398377,-0.097318,0.000000,-0.141175,-0.008223,Caudate
11881,0.255320,0.154169,0.058127,-0.056648,-0.196866,-0.103667,0.064850,-0.124610,0.057833,0.112414,...,-0.171199,0.101784,-0.164377,0.075108,-0.283147,-0.234968,-0.141175,0.000000,-0.024752,Caudate


...

,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,0.034082,0.031152,0.084741,-0.013851,0.062798,0.130403,-0.023966,0.001035,0.017602,-0.000894
33,0.009270,-0.000215,-0.006491,-0.022294,0.008709,0.038025,0.012173,0.010671,-0.004768,0.022405
Caudate,-0.015025,-0.051361,0.039540,0.007019,0.017006,-0.051424,0.000591,-0.036842,0.018777,-0.008860
FP2,0.038316,0.041514,0.033678,-0.021030,-0.014638,0.022031,0.004594,-0.081153,-0.057541,-0.072985
NAcc,0.027847,0.039272,0.065772,0.009250,0.037286,0.028810,-0.022806,-0.031586,-0.014062,-0.037575
OFC1,0.031680,-0.021411,0.019692,-0.010255,0.067701,-0.024269,-0.060924,-0.033868,0.078591,0.011270
OFC2,0.010414,0.016213,0.035815,-0.012655,0.037436,0.048654,-0.003154,-0.011106,0.016352,-0.003500
OFC3,0.004115,0.002109,0.036247,-0.023097,0.020995,0.021169,-0.023600,0.000003,0.008184,-0.006086
OFC4,0.033042,0.035727,0.055931,0.028406,0.031368,0.039945,-0.015512,0.011913,-0.007593,-0.006705


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,-0.319293,-0.088508,0.082083,0.052073,-0.014627,0.094877,-0.004271,0.103266,0.008339,...,0.110622,-0.045570,-0.013251,0.007225,-0.030955,-0.049239,-0.074313,-0.073265,-0.038487,OFC1
1,-0.319293,0.000000,0.043500,0.162900,-0.196880,0.074363,0.076416,0.033003,0.112004,-0.048535,...,-0.080014,0.033593,0.040193,-0.050948,-0.025003,-0.038776,-0.087547,-0.115715,-0.005411,OFC1
2,-0.088508,0.043500,0.000000,-0.251711,0.140998,-0.045390,-0.088574,-0.235902,0.130767,0.115421,...,-0.119579,0.032371,-0.132885,0.119874,-0.028285,-0.055887,-0.125495,-0.059095,0.098849,OFC1
3,0.082083,0.162900,-0.251711,0.000000,0.018188,-0.128789,-0.109122,0.002000,0.049000,0.036620,...,-0.052873,0.073739,-0.004253,0.109748,-0.011544,0.051544,0.131209,0.066692,0.111172,OFC1
4,0.052073,-0.196880,0.140998,0.018188,0.000000,-0.003897,0.019947,0.219032,0.139883,0.076146,...,0.047441,-0.050101,0.062709,-0.040505,0.068173,0.125197,0.208001,0.075092,-0.007859,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.030955,-0.025003,-0.028285,-0.011544,0.068173,-0.027135,0.006096,-0.012931,0.337139,-0.037214,...,-0.169545,-0.072529,-0.446924,-0.106292,0.000000,-0.014024,-0.434642,-0.438924,-0.171197,Caudate
11879,-0.049239,-0.038776,-0.055887,0.051544,0.125197,-0.005620,0.051752,0.034749,0.189681,0.069434,...,-0.142508,-0.040629,-0.443745,-0.072165,-0.014024,0.000000,-0.271857,-0.297552,-0.195981,Caudate
11880,-0.074313,-0.087547,-0.125495,0.131209,0.208001,0.126408,0.028265,0.053204,-0.069475,0.078648,...,-0.035606,-0.033098,-0.264567,-0.080943,-0.434642,-0.271857,0.000000,-0.081997,-0.210408,Caudate
11881,-0.073265,-0.115715,-0.059095,0.066692,0.075092,0.079530,0.025060,0.068291,-0.077665,-0.046963,...,0.076399,0.041861,0.097178,-0.116263,-0.438924,-0.297552,-0.081997,0.000000,0.014739,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,0.030288,0.001976,0.012662,-0.004382,-0.006326,0.009837,-0.070962,0.025357,0.005149,-0.016596
33,0.022907,0.010577,0.024575,0.012744,0.004217,-0.004113,-0.025464,0.009702,-0.030384,-0.010870
Caudate,-0.008152,0.011014,0.040291,-0.002647,-0.046268,0.021752,-0.007178,-0.033639,0.020548,-0.026925
FP2,0.056109,0.033619,0.000139,0.045266,-0.002516,-0.032233,-0.025529,0.007421,-0.004720,-0.029938
NAcc,-0.026324,-0.018784,0.036958,0.042211,-0.019478,0.018643,-0.032554,-0.032250,-0.009653,-0.029296
OFC1,0.037292,0.013339,0.005837,-0.032492,0.009737,0.006132,0.034018,0.051796,0.073417,0.057924
OFC2,0.005992,0.002408,0.013520,-0.001842,-0.014267,0.000547,-0.004775,0.004130,0.002497,-0.000609
OFC3,0.017360,0.010656,-0.003718,0.009323,0.001927,-0.026040,-0.012943,-0.003723,-0.001828,-0.016903
OFC4,-0.010955,-0.065134,-0.015875,-0.025028,0.031996,0.039949,0.007063,0.034856,0.092866,0.045461


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,0.093976,0.161980,0.050064,-0.119658,0.371333,0.206407,0.190032,0.175163,-0.209877,...,-0.035216,-0.116226,-0.002118,-0.086454,-0.036250,0.014252,0.037440,0.032828,-0.019708,OFC1
1,0.093976,0.000000,0.038494,-0.345180,-0.254667,0.161909,0.023204,0.019159,-0.211460,0.193239,...,-0.177383,-0.119696,-0.156315,-0.064254,-0.288720,-0.198565,-0.110856,-0.033568,-0.121178,OFC1
2,0.161980,0.038494,0.000000,0.120968,-0.244734,0.255129,-0.295128,-0.056211,-0.048018,-0.199012,...,-0.144676,-0.141998,-0.024067,-0.067940,-0.184425,-0.107099,-0.150184,-0.065945,-0.102603,OFC1
3,0.050064,-0.345180,0.120968,0.000000,0.038266,0.048568,0.182611,-0.092691,0.186258,-0.257079,...,-0.100501,-0.031831,-0.016231,-0.006679,-0.091756,-0.061850,-0.126974,-0.049891,-0.088550,OFC1
4,-0.119658,-0.254667,-0.244734,0.038266,0.000000,-0.041625,0.096022,-0.317289,-0.095684,-0.373523,...,0.178116,0.158028,0.069162,0.230966,0.001179,-0.091538,-0.014927,-0.082642,-0.028827,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.036250,-0.288720,-0.184425,-0.091756,0.001179,0.001661,-0.158029,-0.056212,-0.044973,0.032602,...,-0.111223,-0.203109,-0.125513,0.052022,0.000000,-0.091203,-0.099905,0.088810,-0.024941,Caudate
11879,0.014252,-0.198565,-0.107099,-0.061850,-0.091538,0.066721,-0.137841,0.138202,0.006246,-0.018548,...,-0.166214,-0.153740,-0.188002,0.025916,-0.091203,0.000000,-0.098953,-0.033251,-0.104329,Caudate
11880,0.037440,-0.110856,-0.150184,-0.126974,-0.014927,-0.030120,-0.133660,0.055669,0.089853,0.075074,...,-0.186982,-0.087057,-0.179705,0.021096,-0.099905,-0.098953,0.000000,-0.041876,-0.056614,Caudate
11881,0.032828,-0.033568,-0.065945,-0.049891,-0.082642,0.090394,-0.116504,-0.000511,-0.020852,-0.034312,...,0.046413,0.130043,0.050166,0.167790,0.088810,-0.033251,-0.041876,0.000000,-0.012512,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,0.004188,0.007228,-0.017049,-0.010546,0.023972,0.039056,0.009822,0.011146,-0.029301,-0.036592
33,-0.007578,0.040789,-0.002211,-0.004549,-0.004731,0.019570,-0.019207,-0.000672,0.054207,0.045885
Caudate,0.007836,0.000356,-0.017889,0.017353,-0.028018,0.004035,-0.012637,-0.003699,0.074003,0.058823
FP2,-0.015320,-0.018982,-0.007002,-0.015473,0.008385,0.025629,0.026465,0.031930,0.024213,0.009365
NAcc,0.014871,0.013602,-0.018992,0.015503,-0.024465,-0.006472,-0.007703,-0.017209,0.098582,0.081261
OFC1,0.006063,0.011615,0.050433,0.026137,-0.052770,0.054085,0.055844,0.039399,-0.002748,0.016625
OFC2,-0.012680,0.003723,0.001436,-0.026146,0.015028,0.076315,0.032689,0.035374,-0.043430,-0.022228
OFC3,-0.018290,-0.005508,0.002175,-0.006158,0.003370,0.006344,-0.015651,0.010891,0.008971,0.009295
OFC4,-0.025113,-0.042544,-0.008954,-0.004251,0.032749,-0.049194,-0.046985,-0.018173,0.020398,0.007939


In [70]:
calculate_FC_features(subj_p, path_placebo, path_baseline)

,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,0.015875,0.114029,0.027870,-0.150400,0.036162,0.007154,0.097649,-0.012585,0.048021,...,-0.044917,-0.073218,0.147417,-0.092367,0.094463,-0.052182,-0.030136,0.004851,0.181669,OFC1
1,0.015875,0.000000,-0.100736,0.250514,-0.130559,-0.115757,0.094144,0.056667,-0.055105,-0.045644,...,0.005775,0.100880,0.170272,0.103629,-0.067990,-0.140894,-0.107936,-0.100159,0.080753,OFC1
2,0.114029,-0.100736,0.000000,0.017365,0.193895,-0.013490,-0.119257,0.135508,-0.082293,0.045722,...,-0.074295,-0.081182,0.086161,-0.041529,0.059812,0.039739,0.011346,-0.039275,-0.011245,OFC1
3,0.027870,0.250514,0.017365,0.000000,-0.079371,0.139871,0.038222,-0.008881,0.019151,-0.067242,...,-0.110686,-0.022186,-0.091639,-0.044531,0.028190,-0.087758,-0.167908,-0.016223,0.060257,OFC1
4,-0.150400,-0.130559,0.193895,-0.079371,0.000000,-0.104955,0.082714,0.046970,0.285312,-0.180294,...,-0.100429,-0.056350,-0.121778,-0.050710,-0.079575,-0.106633,0.041540,0.006260,0.098074,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,0.094463,-0.067990,0.059812,0.028190,-0.079575,-0.045425,-0.141587,0.299168,-0.086219,0.115342,...,0.225320,0.255222,0.384174,0.387920,0.000000,-0.231617,-0.369471,-0.102641,0.038398,Caudate
11879,-0.052182,-0.140894,0.039739,-0.087758,-0.106633,-0.213208,-0.084657,0.095342,-0.084769,0.138407,...,0.282173,0.399688,0.431217,0.484668,-0.231617,0.000000,-0.043411,-0.056353,0.172904,Caudate
11880,-0.030136,-0.107936,0.011346,-0.167908,0.041540,-0.210376,-0.067768,0.079335,-0.158040,0.000202,...,0.153114,0.336504,0.204256,0.379884,-0.369471,-0.043411,0.000000,-0.031502,0.115415,Caudate
11881,0.004851,-0.100159,-0.039275,-0.016223,0.006260,0.037463,0.042660,0.247021,-0.081159,0.144388,...,0.219057,0.123012,0.214279,0.111942,-0.102641,-0.056353,-0.031502,0.000000,0.149540,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,0.000899,0.005177,-0.000056,0.008112,0.047383,0.087342,0.054937,0.077043,0.037664,-0.002483
33,-0.010121,-0.032677,-0.069133,0.012412,-0.025526,-0.016999,-0.009033,-0.045070,0.015279,0.020397
Caudate,-0.021353,-0.078161,-0.009803,-0.000212,-0.006381,0.054038,-0.017966,0.023257,0.015308,0.085899
FP2,-0.010019,-0.020065,0.021943,-0.020539,0.019852,-0.037462,-0.012114,-0.064186,0.064785,0.079434
NAcc,-0.014835,-0.055561,-0.004030,-0.015850,-0.000416,0.083453,0.014692,0.033291,0.070095,0.085630
OFC1,0.007780,-0.003326,-0.027847,-0.021068,0.018782,-0.032828,0.018519,-0.063394,-0.040397,-0.007895
OFC2,-0.005353,-0.000230,0.012471,0.006361,0.048167,0.055810,0.039592,0.043347,0.059723,0.027426
OFC3,-0.018770,0.013625,0.012589,-0.019311,0.013899,-0.002008,0.010125,-0.006805,0.027033,0.040431
OFC4,-0.028305,-0.019143,-0.044439,-0.040856,-0.058148,-0.127220,-0.058471,-0.127697,-0.024529,0.029872


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,0.190445,0.073194,-0.382220,-0.017201,-0.128124,0.321127,0.091531,-0.137893,0.251791,...,0.239700,0.084973,0.089105,0.178683,0.130843,0.094125,0.049880,0.105687,0.147022,OFC1
1,0.190445,0.000000,-0.004117,0.006863,-0.095641,-0.185468,0.305654,0.269033,-0.237873,0.065406,...,0.071070,0.074173,0.083592,0.199495,0.164109,0.093143,0.125512,0.145542,0.177143,OFC1
2,0.073194,-0.004117,0.000000,-0.295966,-0.179050,-0.211247,0.140288,0.053370,-0.247586,0.083841,...,0.170131,0.162644,0.242097,0.087997,0.302560,0.102229,0.062953,0.046039,0.097345,OFC1
3,-0.382220,0.006863,-0.295966,0.000000,-0.045179,0.127450,-0.306028,-0.266273,0.113085,0.050339,...,-0.262011,0.031076,-0.276254,-0.097303,-0.175430,-0.299827,-0.197032,-0.217566,-0.132291,OFC1
4,-0.017201,-0.095641,-0.179050,-0.045179,0.000000,0.082576,0.050343,-0.072825,-0.121748,0.014504,...,-0.046862,-0.056026,-0.259939,0.087184,0.049044,0.018504,0.090876,0.092566,0.029450,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,0.130843,0.164109,0.302560,-0.175430,0.049044,-0.088932,0.231095,0.135283,-0.052160,0.034655,...,0.249118,0.296832,0.357617,0.382637,0.000000,0.161843,0.133226,0.119733,0.274084,Caudate
11879,0.094125,0.093143,0.102229,-0.299827,0.018504,-0.047173,0.499689,0.070208,-0.076079,0.043673,...,0.196826,0.407393,0.058479,0.472522,0.161843,0.000000,-0.073744,0.031342,0.279878,Caudate
11880,0.049880,0.125512,0.062953,-0.197032,0.090876,-0.007022,0.385711,0.035256,-0.086289,0.059919,...,0.121315,0.320045,-0.084892,0.395603,0.133226,-0.073744,0.000000,-0.022070,0.192431,Caudate
11881,0.105687,0.145542,0.046039,-0.217566,0.092566,-0.005193,0.170758,0.176237,-0.092726,0.039909,...,0.179392,0.153453,0.041129,0.262035,0.119733,0.031342,-0.022070,0.000000,0.149139,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.175242,-0.065026,-0.040738,-0.008329,-0.119782,0.040873,0.024200,0.087303,0.108927,-0.004106
33,-0.004591,0.021196,-0.000845,0.018133,0.009188,0.055282,0.047831,0.032933,0.086738,0.079357
Caudate,0.059314,0.031413,0.002969,-0.049811,0.050893,-0.003979,-0.012370,-0.011925,-0.045537,-0.016711
FP2,-0.030809,-0.040968,-0.029256,0.002991,-0.022376,0.065653,0.028090,0.074127,0.090885,0.044873
NAcc,0.056581,0.052668,0.018027,-0.041989,0.083798,-0.048303,-0.070187,-0.022182,-0.050933,-0.009249
OFC1,-0.157195,-0.053081,-0.027668,-0.004928,-0.108266,0.027100,0.013223,0.090809,0.119267,0.034100
OFC2,-0.149772,-0.052955,-0.016973,-0.007719,-0.124767,0.021320,-0.003279,0.067977,0.118051,0.031141
OFC3,-0.048128,-0.024251,-0.012005,0.012614,-0.039876,0.037761,0.056736,0.055995,0.086711,0.038659
OFC4,0.005458,-0.007155,-0.024073,0.021759,0.012300,0.065189,0.044331,0.056898,0.038251,0.034545


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,0.209254,0.138263,-0.077106,-0.086692,0.350376,0.064010,0.175183,-0.170082,-0.126574,...,0.026127,-0.157312,0.106827,0.062214,-0.209493,-0.126396,-0.168360,-0.161174,-0.189616,OFC1
1,0.209254,0.000000,0.136060,-0.233909,-0.264844,0.103377,-0.249077,0.162906,0.013245,-0.123034,...,0.142861,-0.036620,0.045765,0.069258,-0.081858,0.001733,0.005679,0.015059,0.119616,OFC1
2,0.138263,0.136060,0.000000,-0.200235,0.103948,-0.079920,-0.127688,-0.107395,0.099481,-0.000658,...,0.104970,0.216934,0.014585,0.143908,-0.112656,-0.037126,-0.082403,-0.161851,-0.081069,OFC1
3,-0.077106,-0.233909,-0.200235,0.000000,-0.006457,-0.118192,-0.090689,-0.150424,0.202598,0.045969,...,0.219948,0.276879,0.044812,0.056295,-0.159486,-0.097349,-0.151586,-0.223945,-0.049596,OFC1
4,-0.086692,-0.264844,0.103948,-0.006457,0.000000,-0.040100,0.001737,-0.175680,0.041557,0.163982,...,-0.103048,0.189269,-0.148070,0.005978,-0.121989,-0.160590,-0.168593,-0.114030,0.007984,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.209493,-0.081858,-0.112656,-0.159486,-0.121989,-0.055122,-0.051302,-0.263793,-0.174455,0.003471,...,-0.109389,0.278199,-0.255198,-0.075540,0.000000,-0.060233,-0.192481,-0.165582,0.029523,Caudate
11879,-0.126396,0.001733,-0.037126,-0.097349,-0.160590,-0.049692,-0.078760,-0.113561,-0.074547,-0.060128,...,-0.120764,0.279168,-0.235057,-0.133967,-0.060233,0.000000,-0.066734,-0.189435,-0.048754,Caudate
11880,-0.168360,0.005679,-0.082403,-0.151586,-0.168593,0.002426,-0.066681,-0.122519,-0.011628,0.051533,...,-0.022122,0.391345,-0.141955,0.092768,-0.192481,-0.066734,0.000000,-0.097329,-0.019001,Caudate
11881,-0.161174,0.015059,-0.161851,-0.223945,-0.114030,0.041051,0.002276,-0.208610,0.016684,0.047907,...,0.138513,0.493792,0.042458,0.217038,-0.165582,-0.189435,-0.097329,0.000000,0.105968,Caudate


...

,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.020826,-0.009467,0.041133,-0.113702,-0.031101,-0.003203,-0.069650,-0.048374,-0.091981,-0.090212
33,-0.052918,-0.065806,0.014652,-0.069055,-0.103003,-0.028878,-0.029192,-0.034402,-0.043853,-0.038953
Caudate,-0.018114,-0.032006,0.012445,-0.018850,-0.022066,0.014770,0.004473,0.032206,0.032819,0.016719
FP2,-0.015940,-0.017876,-0.001638,-0.027892,-0.026838,-0.033857,-0.051688,-0.039177,-0.033266,-0.034716
NAcc,-0.064556,-0.023699,0.000370,-0.014869,0.001341,0.002196,-0.024461,-0.005285,-0.044616,-0.022394
OFC1,0.045973,0.085256,-0.062037,0.122614,0.144647,0.039600,0.019028,0.083539,0.046164,0.045424
OFC2,0.018418,0.002357,0.002885,0.018601,0.045494,-0.030316,-0.014122,-0.031288,-0.064754,-0.038441
OFC3,0.004933,-0.019701,-0.013763,0.011149,0.019674,0.004885,-0.007427,0.024954,-0.012607,0.000307
OFC4,-0.022086,-0.002399,0.016802,-0.043270,0.004429,-0.005757,-0.030698,-0.010049,-0.012743,-0.009930


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,-0.334015,-0.236028,-0.277050,-0.273890,-0.163530,-0.067208,-0.029536,0.136338,0.039088,...,0.006782,-0.078565,-0.048388,-0.030981,0.002491,-0.049895,-0.098967,0.021787,-0.026176,OFC1
1,-0.334015,0.000000,0.085323,0.121247,0.249504,0.089702,-0.088559,-0.130615,0.039718,0.094179,...,0.053175,0.004951,0.063574,0.047709,0.106311,0.238972,0.201646,-0.013404,-0.086225,OFC1
2,-0.236028,0.085323,0.000000,-0.049598,0.102043,-0.079033,-0.065920,0.051903,-0.124286,-0.223342,...,-0.014668,-0.114612,0.029437,-0.123409,0.008460,-0.058051,-0.184102,-0.125348,0.003549,OFC1
3,-0.277050,0.121247,-0.049598,0.000000,0.089149,-0.010575,-0.121120,0.101367,-0.247271,-0.209339,...,0.074189,0.067056,0.060554,0.117758,0.044129,0.051080,0.137672,0.119243,0.085160,OFC1
4,-0.273890,0.249504,0.102043,0.089149,0.000000,0.072465,-0.228233,-0.039324,-0.119510,0.210930,...,0.189102,0.026174,0.137869,0.040342,0.037963,0.075340,0.172580,0.074773,-0.085554,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,0.002491,0.106311,0.008460,0.044129,0.037963,-0.049665,-0.002559,-0.153674,-0.101434,-0.021747,...,-0.051033,0.035580,-0.123716,0.097808,0.000000,0.024234,-0.072212,-0.006983,0.017608,Caudate
11879,-0.049895,0.238972,-0.058051,0.051080,0.075340,-0.085440,-0.170992,-0.112906,-0.149945,-0.080030,...,0.176236,0.031941,0.035534,0.125925,0.024234,0.000000,-0.122098,0.096415,0.144655,Caudate
11880,-0.098967,0.201646,-0.184102,0.137672,0.172580,-0.025212,-0.156570,-0.035543,-0.030161,-0.076333,...,0.122412,-0.053539,0.018723,-0.042646,-0.072212,-0.122098,0.000000,0.156736,0.055308,Caudate
11881,0.021787,-0.013404,-0.125348,0.119243,0.074773,0.024496,-0.079464,0.013919,-0.051664,-0.057070,...,0.003136,-0.046101,-0.074349,-0.056779,-0.006983,0.096415,0.156736,0.000000,-0.039980,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,0.017573,-0.012717,0.013650,0.035803,0.009655,-0.039672,-0.014805,0.015498,-0.003616,0.009857
33,0.014854,0.029830,-0.002169,0.012338,0.014282,-0.010075,0.006964,0.007361,0.002764,-0.004271
Caudate,-0.022501,-0.018948,-0.010255,-0.021900,-0.010557,-0.013710,-0.009353,-0.012153,0.015660,-0.012736
FP2,-0.007851,0.030701,-0.019556,-0.002828,0.020360,-0.003341,-0.017148,-0.005560,-0.000291,0.035765
NAcc,0.003533,-0.019718,-0.013987,-0.003615,0.004549,0.021795,0.021791,-0.000960,0.006252,0.014965
OFC1,-0.022559,-0.014249,0.008265,-0.012964,-0.022062,0.005182,-0.006254,-0.044673,-0.021336,0.003653
OFC2,0.001763,0.009234,0.005497,0.006584,-0.008481,-0.009779,0.002616,-0.010784,-0.020549,-0.008929
OFC3,0.012666,0.010417,0.001586,0.014205,0.021726,-0.014508,-0.003063,-0.001599,-0.011587,-0.018153
OFC4,-0.014503,0.010643,0.006066,-0.012512,0.002579,-0.046542,-0.013134,-0.026710,-0.036038,-0.022983


,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,Label
0,0.000000,-0.035172,0.041819,-0.031910,0.012674,-0.104426,-0.051943,-0.002854,-0.092623,0.090639,...,-0.092653,-0.040177,-0.039621,-0.050726,0.000137,-0.178244,-0.185576,0.017151,0.131889,OFC1
1,-0.035172,0.000000,0.424580,0.006225,0.055263,0.067423,-0.169632,-0.097845,-0.096634,0.051801,...,-0.081372,-0.108562,-0.052594,-0.095510,0.056715,-0.017741,-0.068797,-0.036313,-0.045338,OFC1
2,0.041819,0.424580,0.000000,0.236337,0.032781,0.122878,-0.098774,-0.116882,-0.121241,-0.008315,...,-0.085637,-0.100198,-0.079983,-0.092142,-0.096756,-0.148098,-0.070963,-0.221526,-0.210503,OFC1
3,-0.031910,0.006225,0.236337,0.000000,0.094638,0.070199,-0.074856,0.102249,0.012742,0.049401,...,0.069229,-0.035274,0.050983,-0.052092,0.049069,0.069413,0.019157,-0.033630,-0.092748,OFC1
4,0.012674,0.055263,0.032781,0.094638,0.000000,0.133429,0.217513,0.212973,-0.069226,0.164213,...,-0.034690,0.054635,-0.081852,0.048001,0.024225,0.177390,-0.039549,0.004051,0.133844,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,0.000137,0.056715,-0.096756,0.049069,0.024225,-0.042178,0.040279,0.057530,0.146833,0.033634,...,0.002130,0.007675,0.012320,0.121349,0.000000,0.069385,-0.084088,-0.101951,0.020477,Caudate
11879,-0.178244,-0.017741,-0.148098,0.069413,0.177390,-0.055191,0.063088,0.149463,0.131914,0.088277,...,-0.066450,0.003775,0.065023,0.031693,0.069385,0.000000,-0.017897,0.016925,0.032019,Caudate
11880,-0.185576,-0.068797,-0.070963,0.019157,-0.039549,0.026194,0.057109,0.160042,0.084898,0.015825,...,-0.032875,0.110155,-0.040403,0.086180,-0.084088,-0.017897,0.000000,-0.050339,0.021322,Caudate
11881,0.017151,-0.036313,-0.221526,-0.033630,0.004051,0.079314,0.071868,0.034095,0.051861,-0.047720,...,-0.078050,0.053136,0.018256,0.076139,-0.101951,0.016925,-0.050339,0.000000,0.136102,Caudate


,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,0.006074,-0.007116,0.004398,-0.046493,-0.001339,0.009297,0.040254,0.001502,-0.003279,0.022000
33,-0.003312,0.006418,0.012586,-0.037832,-0.031743,-0.001171,0.037597,-0.001493,0.004994,0.009712
Caudate,-0.022772,0.001077,-0.005257,0.000837,0.000795,-0.026403,0.008607,-0.005666,-0.001204,-0.007191
FP2,0.014062,-0.000445,-0.047042,-0.017412,-0.009239,-0.003782,-0.002114,0.012358,-0.016937,-0.048485
NAcc,0.033941,0.025830,0.015324,0.020260,-0.050739,-0.014888,-0.004719,-0.005598,-0.025530,-0.017219
OFC1,-0.017164,-0.052430,-0.012495,-0.048745,-0.002476,0.027600,0.056891,0.012804,-0.032236,-0.033052
OFC2,0.002065,-0.004225,0.012297,-0.017978,-0.011529,0.013846,0.037577,-0.003347,-0.012337,-0.012023
OFC3,-0.003283,-0.001637,0.007803,-0.014545,-0.007273,0.010358,0.025187,-0.001930,-0.007589,-0.003075
OFC4,-0.038801,-0.013755,0.015152,0.007496,0.008398,0.020404,-0.011438,0.013137,-0.023342,-0.008272


In [43]:
subj_diff_conc =pd.concat([curr_subj_df_1, labels['Label']], axis=1)


0        1.000000
1        0.111325
2        0.085908
3       -0.045321
4       -0.004807
           ...   
11878   -0.168262
11879   -0.135171
11880   -0.120086
11881   -0.044072
11882    0.007843
Name: 0, Length: 11883, dtype: float64

In [26]:
curr_subj_df_1 = pd.read_csv('C:/Users/canlab/Desktop/Masterthesis_python/ket/corr_mat_all_ses-1h_sub-bg52.csv', header=None)


In [27]:
curr_subj_df_1

,0,1,2,3,4,5,6,7,8,9,...,11873,11874,11875,11876,11877,11878,11879,11880,11881,11882
0,1.000000,0.111325,0.085908,-0.045321,-0.004807,-0.099480,0.170437,-0.048781,0.019528,-0.031685,...,-0.058467,-0.075202,-0.064980,-0.028280,-0.069227,-0.168262,-0.135171,-0.120086,-0.044072,0.007843
1,0.111325,1.000000,0.511422,-0.027034,0.110320,-0.123253,0.274396,-0.110925,0.120021,0.207649,...,-0.005069,0.001031,-0.079389,0.077031,-0.084573,-0.015426,-0.052844,-0.062932,-0.134808,-0.072376
2,0.085908,0.511422,1.000000,0.366440,0.030647,-0.153734,0.262213,-0.001478,0.112283,0.092765,...,0.002157,0.030943,0.034259,0.047480,-0.003531,-0.020513,-0.053759,-0.124409,-0.122028,-0.094261
3,-0.045321,-0.027034,0.366440,1.000000,0.532926,0.025374,-0.158213,0.093456,0.054608,-0.047442,...,-0.085327,0.079361,0.027758,0.073975,0.069671,-0.057160,0.049659,-0.057933,-0.061688,0.001319
4,-0.004807,0.110320,0.030647,0.532926,1.000000,0.013608,-0.033499,-0.033688,0.006741,0.015662,...,-0.044621,0.026989,0.000859,0.040498,0.027264,-0.060484,0.014331,-0.014204,-0.107151,-0.049634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.168262,-0.015426,-0.020513,-0.057160,-0.060484,-0.144416,0.043892,-0.044705,0.178726,-0.009200,...,0.381255,0.290254,0.264194,0.275315,0.047015,1.000000,0.614231,0.347697,0.339942,0.133109
11879,-0.135171,-0.052844,-0.053759,0.049659,0.014331,-0.167295,0.007226,-0.080090,0.039901,-0.027869,...,0.399164,0.265478,0.277005,0.190351,0.104961,0.614231,1.000000,0.678077,0.323681,0.233135
11880,-0.120086,-0.062932,-0.124409,-0.057933,-0.014204,-0.137836,0.062362,-0.132397,-0.007235,-0.012748,...,0.443975,0.270375,0.277627,0.154443,0.065119,0.347697,0.678077,1.000000,0.629508,0.180612
11881,-0.044072,-0.134808,-0.122028,-0.061688,-0.107151,-0.179118,-0.061816,-0.092567,0.068319,-0.045464,...,0.332474,0.347050,0.281744,0.185829,0.108280,0.339942,0.323681,0.629508,1.000000,0.518147


In [25]:
curr_subj_df

,0,1,2,3,4,5,6,7,8,9,...,11873,11874,11875,11876,11877,11878,11879,11880,11881,11882
0,1.000000,-0.016530,0.020265,0.153289,-0.032359,-0.043095,-0.046219,0.061790,0.030660,0.020752,...,0.029662,-0.006105,-0.046263,0.038751,-0.026752,-0.047304,-0.057025,0.075896,0.079893,0.011055
1,-0.016530,1.000000,0.328585,0.174994,0.154327,0.072323,0.015999,0.048026,-0.048163,-0.078161,...,0.038657,-0.094809,-0.032011,-0.105345,-0.025568,-0.111525,-0.048393,0.046235,0.009696,-0.029768
2,0.020265,0.328585,1.000000,0.291534,0.084999,-0.097101,-0.023530,-0.022142,-0.166702,-0.053509,...,0.046382,0.010156,0.053461,-0.172389,0.128015,0.066731,0.066211,0.015231,0.006631,0.026776
3,0.153289,0.174994,0.291534,1.000000,0.278441,-0.029912,0.027076,0.115411,-0.106525,-0.097499,...,-0.032622,-0.090323,0.039216,-0.063383,0.035601,0.002338,-0.078718,-0.050652,-0.013799,0.041243
4,-0.032359,0.154327,0.084999,0.278441,1.000000,0.075166,-0.201466,0.108222,-0.051817,-0.195235,...,0.045302,-0.095331,0.064022,-0.093833,0.002523,-0.064294,-0.019641,0.012186,-0.033444,-0.018951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.047304,-0.111525,0.066731,0.002338,-0.064294,0.056861,-0.038732,-0.058585,0.020935,0.039992,...,0.225401,0.249209,0.169913,0.125348,0.093278,1.000000,0.609041,0.306435,0.363554,0.235825
11879,-0.057025,-0.048393,0.066211,-0.078718,-0.019641,0.082090,-0.097020,-0.021101,0.011533,0.049635,...,0.355342,0.333630,0.115295,0.273481,0.081769,0.609041,1.000000,0.690599,0.385951,0.248285
11880,0.075896,0.046235,0.015231,-0.050652,0.012186,0.093034,-0.077893,-0.085217,0.016018,0.034789,...,0.466264,0.340582,0.249403,0.250092,0.140824,0.306435,0.690599,1.000000,0.733906,0.271069
11881,0.079893,0.009696,0.006631,-0.013799,-0.033444,0.063754,-0.037203,-0.091869,0.052345,0.074618,...,0.444480,0.261497,0.305324,0.159001,0.171126,0.363554,0.385951,0.733906,1.000000,0.623690


In [30]:
input_df = curr_subj_df_1.subtract(curr_subj_df)

In [24]:
curr_subj_df = pd.read_csv('C:/Users/canlab/Desktop/Masterthesis_python/baseline/corr_mat_all_ses-baseline_sub-bg52.csv', header=None)


In [33]:
df = pd.concat([input_df, labels], axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,11875,11876,11877,11878,11879,11880,11881,11882,ID,Label
0,0.000000,0.127854,0.065643,-0.198610,0.027552,-0.056385,0.216656,-0.110571,-0.011132,-0.052437,...,-0.018717,-0.067031,-0.042474,-0.120957,-0.078146,-0.195982,-0.123966,-0.003212,205889,OFC1
1,0.127854,0.000000,0.182837,-0.202029,-0.044006,-0.195576,0.258397,-0.158951,0.168185,0.285810,...,-0.047379,0.182377,-0.059005,0.096099,-0.004451,-0.109167,-0.144505,-0.042608,205890,OFC1
2,0.065643,0.182837,0.000000,0.074905,-0.054351,-0.056633,0.285743,0.020664,0.278985,0.146274,...,-0.019202,0.219870,-0.131546,-0.087244,-0.119970,-0.139639,-0.128659,-0.121037,205980,OFC1
3,-0.198610,-0.202029,0.074905,0.000000,0.254485,0.055286,-0.185289,-0.021955,0.161133,0.050057,...,-0.011458,0.137358,0.034070,-0.059499,0.128377,-0.007281,-0.047889,-0.039924,206071,OFC1
4,0.027552,-0.044006,-0.054351,0.254485,0.000000,-0.061558,0.167968,-0.141910,0.058558,0.210897,...,-0.063164,0.134331,0.024741,0.003810,0.033972,-0.026390,-0.073707,-0.030683,206162,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.120957,0.096099,-0.087244,-0.059499,0.003810,-0.201277,0.082623,0.013880,0.157791,-0.049192,...,0.094281,0.149966,-0.046263,0.000000,0.005190,0.041262,-0.023612,-0.102716,491091,Caudate
11879,-0.078146,-0.004451,-0.119970,0.128377,0.033972,-0.249385,0.104246,-0.058989,0.028369,-0.077504,...,0.161711,-0.083131,0.023192,0.005190,0.000000,-0.012522,-0.062270,-0.015150,491182,Caudate
11880,-0.195982,-0.109167,-0.139639,-0.007281,-0.026390,-0.230870,0.140254,-0.047179,-0.023253,-0.047538,...,0.028224,-0.095649,-0.075706,0.041262,-0.012522,0.000000,-0.104398,-0.090457,491273,Caudate
11881,-0.123966,-0.144505,-0.128659,-0.047889,-0.073707,-0.242871,-0.024613,-0.000698,0.015974,-0.120083,...,-0.023581,0.026828,-0.062846,-0.023612,-0.062270,-0.104398,0.000000,-0.105543,491364,Caudate


In [53]:
new_df = median[[4877, 5373, 5379, 4870, 4257, 2331, 2335, 2304, 2332, 2359]].copy()

In [54]:
new_df

,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.060794,-0.039162,-0.030193,-0.030875,-0.006244,-0.014368,0.109128,-0.069767,0.002359,-0.037030
33,-0.016118,-0.010245,-0.008499,-0.015682,-0.025337,-0.018579,0.016135,0.019050,-0.019523,0.010395
Caudate,0.018753,-0.004139,-0.005690,0.010057,-0.006822,-0.001468,-0.045082,0.024679,0.032477,-0.012345
FP2,0.041885,-0.040878,0.020763,0.027150,0.022503,-0.061360,-0.057410,0.040132,-0.044452,-0.039697
NAcc,-0.038510,-0.050074,-0.025111,-0.009234,0.024019,0.023853,0.017390,-0.001289,0.024356,-0.024291
OFC1,-0.095958,-0.063177,0.011704,-0.090524,0.007438,0.051886,0.117975,-0.033560,-0.025783,-0.001090
OFC2,-0.009966,-0.020165,0.011647,-0.033929,-0.009588,0.008262,0.064357,-0.018491,-0.016428,-0.017696
OFC3,-0.020115,-0.026960,-0.001697,-0.026784,-0.002077,0.016961,0.052406,-0.012243,-0.004439,-0.009416
OFC4,-0.071011,-0.023258,-0.043145,-0.025431,0.019256,0.048199,0.058971,-0.048058,0.005126,-0.028535


In [56]:
new_df

,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.060794,-0.039162,-0.030193,-0.030875,-0.006244,-0.014368,0.109128,-0.069767,0.002359,-0.037030
33,-0.016118,-0.010245,-0.008499,-0.015682,-0.025337,-0.018579,0.016135,0.019050,-0.019523,0.010395
Caudate,0.018753,-0.004139,-0.005690,0.010057,-0.006822,-0.001468,-0.045082,0.024679,0.032477,-0.012345
FP2,0.041885,-0.040878,0.020763,0.027150,0.022503,-0.061360,-0.057410,0.040132,-0.044452,-0.039697
NAcc,-0.038510,-0.050074,-0.025111,-0.009234,0.024019,0.023853,0.017390,-0.001289,0.024356,-0.024291
OFC1,-0.095958,-0.063177,0.011704,-0.090524,0.007438,0.051886,0.117975,-0.033560,-0.025783,-0.001090
OFC2,-0.009966,-0.020165,0.011647,-0.033929,-0.009588,0.008262,0.064357,-0.018491,-0.016428,-0.017696
OFC3,-0.020115,-0.026960,-0.001697,-0.026784,-0.002077,0.016961,0.052406,-0.012243,-0.004439,-0.009416
OFC4,-0.071011,-0.023258,-0.043145,-0.025431,0.019256,0.048199,0.058971,-0.048058,0.005126,-0.028535


In [57]:
median

,0,1,2,3,4,5,6,7,8,9,...,11875,11876,11877,11878,11879,11880,11881,11882,ID,Label
0,0.000000,0.127854,0.065643,-0.198610,0.027552,-0.056385,0.216656,-0.110571,-0.011132,-0.052437,...,-0.018717,-0.067031,-0.042474,-0.120957,-0.078146,-0.195982,-0.123966,-0.003212,205889,OFC1
1,0.127854,0.000000,0.182837,-0.202029,-0.044006,-0.195576,0.258397,-0.158951,0.168185,0.285810,...,-0.047379,0.182377,-0.059005,0.096099,-0.004451,-0.109167,-0.144505,-0.042608,205890,OFC1
2,0.065643,0.182837,0.000000,0.074905,-0.054351,-0.056633,0.285743,0.020664,0.278985,0.146274,...,-0.019202,0.219870,-0.131546,-0.087244,-0.119970,-0.139639,-0.128659,-0.121037,205980,OFC1
3,-0.198610,-0.202029,0.074905,0.000000,0.254485,0.055286,-0.185289,-0.021955,0.161133,0.050057,...,-0.011458,0.137358,0.034070,-0.059499,0.128377,-0.007281,-0.047889,-0.039924,206071,OFC1
4,0.027552,-0.044006,-0.054351,0.254485,0.000000,-0.061558,0.167968,-0.141910,0.058558,0.210897,...,-0.063164,0.134331,0.024741,0.003810,0.033972,-0.026390,-0.073707,-0.030683,206162,OFC1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11878,-0.120957,0.096099,-0.087244,-0.059499,0.003810,-0.201277,0.082623,0.013880,0.157791,-0.049192,...,0.094281,0.149966,-0.046263,0.000000,0.005190,0.041262,-0.023612,-0.102716,491091,Caudate
11879,-0.078146,-0.004451,-0.119970,0.128377,0.033972,-0.249385,0.104246,-0.058989,0.028369,-0.077504,...,0.161711,-0.083131,0.023192,0.005190,0.000000,-0.012522,-0.062270,-0.015150,491182,Caudate
11880,-0.195982,-0.109167,-0.139639,-0.007281,-0.026390,-0.230870,0.140254,-0.047179,-0.023253,-0.047538,...,0.028224,-0.095649,-0.075706,0.041262,-0.012522,0.000000,-0.104398,-0.090457,491273,Caudate
11881,-0.123966,-0.144505,-0.128659,-0.047889,-0.073707,-0.242871,-0.024613,-0.000698,0.015974,-0.120083,...,-0.023581,0.026828,-0.062846,-0.023612,-0.062270,-0.104398,0.000000,-0.105543,491364,Caudate


In [55]:
median=df

In [48]:
split_df = df.groupby('Label')
median=split_df.median()
median

,0,1,2,3,4,5,6,7,8,9,...,11874,11875,11876,11877,11878,11879,11880,11881,11882,ID
Label,,,,,,,,,,,,,,,,,,,,,
25,0.013648,0.017270,0.065304,-0.051433,-0.112115,-0.066033,0.099286,-0.070439,0.056621,0.048587,...,-0.007378,-0.039570,-0.020021,-0.042742,0.008140,-0.035263,-0.042805,-0.015419,-0.016945,303801.5
33,-0.005646,-0.005258,0.029123,-0.010192,-0.021238,0.003132,0.015740,0.011056,-0.019490,0.006860,...,0.002595,0.008243,-0.011199,0.012696,-0.001176,0.024042,0.029684,0.026246,0.023820,453545.0
Caudate,-0.018717,-0.062439,-0.048859,0.007331,0.040747,-0.029810,-0.044487,-0.036361,0.020127,0.004836,...,0.006568,0.028351,0.001136,0.033673,0.001459,-0.015109,-0.041268,-0.025517,-0.004670,413088.0
FP2,0.011955,-0.032739,-0.011977,-0.138010,-0.029001,-0.031327,0.112272,-0.125528,0.026712,0.046011,...,0.029297,0.061134,0.009543,0.046120,-0.015545,-0.023795,0.017531,0.023492,0.041648,346030.5
NAcc,-0.002301,-0.037687,0.037154,-0.009040,-0.014570,-0.038954,0.018758,-0.047390,0.024256,0.005363,...,0.027552,0.028319,0.016045,0.037180,0.063548,0.045356,0.002658,0.035605,0.011935,333375.0
OFC1,0.020181,0.063723,0.081433,-0.018091,-0.010146,-0.128520,0.122702,-0.052926,0.008500,-0.027575,...,0.026170,0.013820,0.034637,-0.082629,0.023839,0.018175,0.007143,-0.010896,-0.007583,245375.5
OFC2,0.014852,-0.002298,0.034877,-0.030563,-0.056379,-0.066373,0.080974,-0.060242,0.016271,0.027073,...,0.014770,-0.003812,0.019297,-0.010520,0.020869,0.004433,-0.001279,-0.003353,0.006079,254761.0
OFC3,0.021372,0.003180,0.024466,-0.034334,-0.040410,-0.049456,0.067607,-0.040370,0.022379,0.033201,...,-0.013419,-0.012629,-0.005604,-0.018807,0.008208,-0.004951,-0.012299,-0.010948,-0.014903,264934.5
OFC4,0.022246,0.036206,0.038038,0.013647,-0.011318,-0.084387,0.006875,-0.060238,0.000320,-0.003079,...,-0.000484,-0.010242,0.035634,-0.049147,0.017554,0.015546,0.015974,0.012991,-0.001669,295507.0


In [47]:
median[0]

Label
25         0.013648
33        -0.005646
Caudate   -0.018717
FP2        0.011955
NAcc      -0.002301
OFC1       0.020181
OFC2       0.014852
OFC3       0.021372
OFC4       0.022246
OFC5       0.034404
OFC6       0.021045
OFC7      -0.016812
Putamen    0.014742
p24ab      0.061167
p24c       0.022664
p32        0.036881
s24        0.035766
s32        0.057154
Name: 0, dtype: float64

In [40]:
labels

,ID,Label
0,205889,OFC1
1,205890,OFC1
2,205980,OFC1
3,206071,OFC1
4,206162,OFC1
...,...,...
11878,491091,Caudate
11879,491182,Caudate
11880,491273,Caudate
11881,491364,Caudate


In [37]:
labels.iloc[0]

ID       205889
Label      OFC1
Name: 0, dtype: object

In [41]:
roi = labels['Label']

In [42]:
roi

0           OFC1
1           OFC1
2           OFC1
3           OFC1
4           OFC1
          ...   
11878    Caudate
11879    Caudate
11880    Caudate
11881    Caudate
11882    Caudate
Name: Label, Length: 11883, dtype: object

In [72]:
ex = np.load('C:/Users/canlab/Desktop/Masterthesis_python/placebo/FC_features-xp38', allow_pickle=True)

In [73]:
ex

,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,0.006074,-0.007116,0.004398,-0.046493,-0.001339,0.009297,0.040254,0.001502,-0.003279,0.022000
33,-0.003312,0.006418,0.012586,-0.037832,-0.031743,-0.001171,0.037597,-0.001493,0.004994,0.009712
Caudate,-0.022772,0.001077,-0.005257,0.000837,0.000795,-0.026403,0.008607,-0.005666,-0.001204,-0.007191
FP2,0.014062,-0.000445,-0.047042,-0.017412,-0.009239,-0.003782,-0.002114,0.012358,-0.016937,-0.048485
NAcc,0.033941,0.025830,0.015324,0.020260,-0.050739,-0.014888,-0.004719,-0.005598,-0.025530,-0.017219
OFC1,-0.017164,-0.052430,-0.012495,-0.048745,-0.002476,0.027600,0.056891,0.012804,-0.032236,-0.033052
OFC2,0.002065,-0.004225,0.012297,-0.017978,-0.011529,0.013846,0.037577,-0.003347,-0.012337,-0.012023
OFC3,-0.003283,-0.001637,0.007803,-0.014545,-0.007273,0.010358,0.025187,-0.001930,-0.007589,-0.003075
OFC4,-0.038801,-0.013755,0.015152,0.007496,0.008398,0.020404,-0.011438,0.013137,-0.023342,-0.008272


In [74]:
exk = np.load('C:/Users/canlab/Desktop/Masterthesis_python/ket/FC_features-bg52', allow_pickle=True)

In [75]:
exk

,4877,5373,5379,4870,4257,2331,2335,2304,2332,2359
Label,,,,,,,,,,
25,-0.060794,-0.039162,-0.030193,-0.030875,-0.006244,-0.014368,0.109128,-0.069767,0.002359,-0.037030
33,-0.016118,-0.010245,-0.008499,-0.015682,-0.025337,-0.018579,0.016135,0.019050,-0.019523,0.010395
Caudate,0.018753,-0.004139,-0.005690,0.010057,-0.006822,-0.001468,-0.045082,0.024679,0.032477,-0.012345
FP2,0.041885,-0.040878,0.020763,0.027150,0.022503,-0.061360,-0.057410,0.040132,-0.044452,-0.039697
NAcc,-0.038510,-0.050074,-0.025111,-0.009234,0.024019,0.023853,0.017390,-0.001289,0.024356,-0.024291
OFC1,-0.095958,-0.063177,0.011704,-0.090524,0.007438,0.051886,0.117975,-0.033560,-0.025783,-0.001090
OFC2,-0.009966,-0.020165,0.011647,-0.033929,-0.009588,0.008262,0.064357,-0.018491,-0.016428,-0.017696
OFC3,-0.020115,-0.026960,-0.001697,-0.026784,-0.002077,0.016961,0.052406,-0.012243,-0.004439,-0.009416
OFC4,-0.071011,-0.023258,-0.043145,-0.025431,0.019256,0.048199,0.058971,-0.048058,0.005126,-0.028535
